# Using virtualizarr in the cloud

In [17]:
import earthaccess
import xarray as xr
import fsspec

In [2]:
from virtualizarr import open_virtual_dataset

In [3]:
earthaccess.login()

Enter your Earthdata Login username:  deanh808
Enter your Earthdata password:  ········


## Try with the *MUR25-JPL-L4-GLOB-v04.2* collection

In [4]:
granule_metadata = earthaccess.search_data(short_name="MUR25-JPL-L4-GLOB-v04.2", count=100, cloud_hosted=True)

In [5]:
fileobjs = earthaccess.open(granule_metadata)

QUEUEING TASKS | :   0%|          | 0/100 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/100 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
s3paths = [f.full_name for f in fileobjs]
s3paths[0]

's3://podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2/20020901090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc'

**dmrpp file paths can be obtained by appending "dmrpp" to the granule paths**

In [7]:
paths_dmrpp = [p+".dmrpp" for p in s3paths]
paths_dmrpp[0]

's3://podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2/20020901090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc.dmrpp'

In [31]:
fs_s3 = earthaccess.get_s3fs_session(daac="PODAAC")

In [34]:
%%time
virtual_datasets = [
    open_virtual_dataset(p, indexes={}, filetype="dmrpp", reader_options={"storage_options": fs_s3.storage_options})
    for p in paths_dmrpp
]

CPU times: user 582 ms, sys: 20.8 ms, total: 602 ms
Wall time: 6.16 s


In [35]:
virtual_datasets[0]

<xarray.Dataset> Size: 10MB
Dimensions:           (time: 1, lat: 720, lon: 1440)
Coordinates:
    time              (time) int32 4B ManifestArray<shape=(1,), dtype=int32, ...
    lat               (lat) float32 3kB ManifestArray<shape=(720,), dtype=flo...
    lon               (lon) float32 6kB ManifestArray<shape=(1440,), dtype=fl...
Data variables:
    analysed_sst      (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...
    analysis_error    (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...
    mask              (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...
    sea_ice_fraction  (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...
    sst_anomaly       (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...

In [36]:
%%time
virtual_ds_combined = xr.combine_nested(virtual_datasets, concat_dim=['time'], coords='minimal', compat='override', combine_attrs='drop_conflicts')

CPU times: user 27.8 ms, sys: 0 ns, total: 27.8 ms
Wall time: 27.4 ms


In [37]:
virtual_ds_combined

<xarray.Dataset> Size: 1GB
Dimensions:           (time: 100, lat: 720, lon: 1440)
Coordinates:
    time              (time) int32 400B ManifestArray<shape=(100,), dtype=int...
    lat               (lat) float32 3kB ManifestArray<shape=(720,), dtype=flo...
    lon               (lon) float32 6kB ManifestArray<shape=(1440,), dtype=fl...
Data variables:
    analysed_sst      (time, lat, lon) int16 207MB ManifestArray<shape=(100, ...
    analysis_error    (time, lat, lon) int16 207MB ManifestArray<shape=(100, ...
    mask              (time, lat, lon) int16 207MB ManifestArray<shape=(100, ...
    sea_ice_fraction  (time, lat, lon) int16 207MB ManifestArray<shape=(100, ...
    sst_anomaly       (time, lat, lon) int16 207MB ManifestArray<shape=(100, ...

In [38]:
virtual_ds_combined.virtualize.to_kerchunk('combined.json', format='json')

In [39]:
fs_local = fsspec.filesystem('reference', fo='combined.json')
m = fs_local.get_mapper('')

ds = xr.open_dataset(
    m, engine='kerchunk', 
    chunks={}, 
    backend_kwargs={
        "storage_options": {
            "remote_protocol": "s3",
            "remote_options": fs_s3.storage_options,
            }
    })

ReferenceNotReachable: Reference "analysed_sst/0.0.0" failed to fetch target ['s3://podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2/20020901090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc', 60292, 826371]

In [40]:
data = xr.open_dataset(
    "reference://",
    engine="zarr",
    chunks={},
    backend_kwargs={
        "consolidated": False,
        "storage_options": {
            "fo": "combined.json",
            "remote_protocol": "s3",
            "remote_options": fs_s3.storage_options,
        }
    },
)

ValueError: Shuffle buffer is not an integer multiple of elementsize

## Opening netcdf files for comparison

In [26]:
%%time
data_nc = xr.open_mfdataset(earthaccess.open(granule_metadata[:100]))

QUEUEING TASKS | :   0%|          | 0/100 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/100 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 5.34 s, sys: 673 ms, total: 6.01 s
Wall time: 20.7 s


In [5]:
data_nc

<xarray.Dataset>
Dimensions:           (time: 10, lat: 720, lon: 1440)
Coordinates:
  * time              (time) datetime64[ns] 2002-09-01T09:00:00 ... 2002-09-1...
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sst_anomaly       (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/54)
    Conventions:                CF-1.7, ACDD-1.3
    title:                      Daily 0.25-degree MUR SST, Final product
    summary:                    A low-resolution version of the MUR SST analy...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    ...                         ...
    publisher_name:             GHRSST Project Office
    publisher_url:              https://www.ghrsst.org
    publisher_email:            gpc@ghrsst.org
    file_quality_level:         3
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    acknowledgment:             Please acknowledge the use of these data with...